<a href="https://colab.research.google.com/github/Komal-patra/EU_AI_ACT_chatbot_using_T5_Fine_tuned_Model/blob/main/EU_AI_ACT_T5_fine_tuned_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.1 MB/s eta 0:00:00
ERROR: pip's de

In [ ]:
import numpy as np
import os
import pickle
import math

from peft import LoraConfig, get_peft_model
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from google.colab import userdata

In [ ]:
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [ ]:
# Load the tokenizer, model, and data collator
# since the data will keep on expanding we will go with the google/flan-t5-large in the coming time
MODEL_NAME = "google/flan-t5-base"

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME, token=os.environ['HF_TOKEN'])
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, token=os.environ['HF_TOKEN'])
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Specify the path to your pickle file
train_file_path = 'train_dataset_july12.pkl'
val_file_path = 'validation_dataset_july12.pkl'

# Open the pickle file in read-binary mode and load the data
with open(train_file_path, 'rb') as file:
    train_loaded_data = pickle.load(file)
    print('Train file loaded successfully')

with open(val_file_path, 'rb') as file:
    val_loaded_data = pickle.load(file)
    print('Validation file loaded successfully')

Train file loaded successfully
Validation file loaded successfully


In [ ]:
# We prefix our tasks with "answer the question"
prefix = "Please answer this question: "

# Define the preprocessing function

def preprocess_function(examples):
   """Add prefix to the sentences, tokenize the text, and set the labels"""
   # The "inputs" are the tokenized answer:
   inputs = [prefix + doc for doc in examples["Questions"]]
   model_inputs = tokenizer(inputs, max_length=128, truncation=True)

   # The "labels" are the tokenized outputs:
   labels = tokenizer(text_target=examples["Answers"],
                      max_length=512,
                      truncation=True)

   model_inputs["labels"] = labels["input_ids"]
   return model_inputs

In [ ]:
# Tokenize datasets
tokenized_train_dataset = train_loaded_data.map(preprocess_function, batched=True, remove_columns=["Questions", "Answers"]
)

tokenized_eval_dataset = val_loaded_data.map(preprocess_function, batched=True, remove_columns=["Questions", "Answers"]
)


Map:   0%|          | 0/1424 [00:00<?, ? examples/s]

Map:   0%|          | 0/159 [00:00<?, ? examples/s]

In [ ]:
# Configure LoRA
lora_config = LoraConfig(
    r=8,
    target_modules=["k","q","v","o"],
    task_type="SEQ_2_SEQ_LM",
)

model = get_peft_model(model, lora_config)

In [ ]:
BATCH_SIZE = int((len(tokenized_train_dataset) * .8)) / 100


In [ ]:
BATCH_SIZE

11.39

In [ ]:
math.ceil(BATCH_SIZE)


12

In [ ]:
# Global Parameters
L_RATE = 0.001
BATCH_SIZE = int((len(tokenized_train_dataset) * .4)) / 100
BATCH_SIZE = math.ceil(BATCH_SIZE)
gradient_accumulation_steps = 2
WEIGHT_DECAY = 0.0001

STEPS = len(tokenized_train_dataset) / (BATCH_SIZE * gradient_accumulation_steps)
number_of_epochs = 20
MAX_STEPS = int(number_of_epochs * STEPS)
WARM_UP_STEPS = int(0.01 * MAX_STEPS)
LOGGING_STEPS = 50
SAVE_STEPS = 475
SAVE_TOTAL_LIM  = 5

In [ ]:
MAX_STEPS

2373

In [ ]:
# Set up training arguments
training_args = Seq2SeqTrainingArguments(
   output_dir="./results",
   evaluation_strategy="steps",
   learning_rate=L_RATE,
   per_device_train_batch_size=BATCH_SIZE,
   weight_decay=WEIGHT_DECAY,
   save_total_limit=SAVE_TOTAL_LIM,
   predict_with_generate=True,
   push_to_hub=False,
   gradient_accumulation_steps=gradient_accumulation_steps,
   max_steps=MAX_STEPS,
   warmup_steps=WARM_UP_STEPS,
   logging_steps=LOGGING_STEPS,
   eval_steps = LOGGING_STEPS,
   generation_max_length=512,
   save_steps=SAVE_STEPS  # Adding checkpointing

)

In [ ]:
trainer = Seq2SeqTrainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train_dataset,
   eval_dataset=tokenized_eval_dataset,
   tokenizer=tokenizer,
   data_collator=data_collator,
)

In [ ]:
# Initialize parameters
best_eval_loss = float('inf')
no_improvement_counter = 0
patience = 3

In [ ]:
# Training loop
global_step = 0

while global_step < MAX_STEPS:
    # Train the model for one step
    trainer.train(resume_from_checkpoint=False)
    global_step += 1

    # Evaluate on the validation set
    if global_step % training_args.eval_steps == 0:
        eval_results = trainer.evaluate()
        eval_loss = eval_results['eval_loss']

        # Check for improvement
        if eval_loss < best_eval_loss:
            best_eval_loss = eval_loss
            trainer.save_model(f"best_model_step_{global_step}")
            print(f"Checkpoint saved at step {global_step} with loss {eval_loss:.4f}")
            no_improvement_counter = 0
        else:
            no_improvement_counter += 1

        # Check if early stopping criteria are met
        if no_improvement_counter >= patience:
            print(f"No improvement for {patience} evaluations. Stopping training.")
            break

print("Training complete.")
print("Best evaluation result:", best_eval_loss)

Step,Training Loss,Validation Loss
50,3.003700,2.511417
100,2.701900,2.396569
150,2.619500,2.323939
200,2.551600,2.275733
250,2.481500,2.234506
300,2.459000,2.204770
350,2.400600,2.187180
400,2.372900,2.155580
450,2.359100,2.136346
500,2.323400,2.120058


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Step,Training Loss,Validation Loss
50,1.999500,1.984826
100,2.040100,1.967376
150,2.036200,1.967531
200,2.010600,1.956831
250,2.003400,1.957396
300,2.006500,1.959663
350,1.967200,1.947434
400,1.955500,1.946301
450,1.976800,1.936229
500,1.946000,1.934069


Checkpoint destination directory ./results/checkpoint-475 already exists and is non-empty. Saving will proceed but saved results may be invalid.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Checkpoint destination directory ./results/checkpoint-950 already exists and is non-empty. Saving will proceed but saved results may be invalid.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Checkpoint destination directory ./results/checkpoint-1425 already exists and is non-empty. Saving will proceed but saved results may be inval

Step,Training Loss,Validation Loss
50,1.768700,1.905766
100,1.806600,1.895883
150,1.813000,1.900690
200,1.785500,1.898278


KeyboardInterrupt: 

In [ ]:
output_dir = "/content/eu_ai_model_t5_15july"
trainer.save_model(output_dir)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:

!zip -r saved_model.zip eu_ai_model_t5


  adding: eu_ai_model_t5/ (stored 0%)
  adding: eu_ai_model_t5/adapter_config.json (deflated 51%)
  adding: eu_ai_model_t5/added_tokens.json (deflated 83%)
  adding: eu_ai_model_t5/spiece.model (deflated 48%)
  adding: eu_ai_model_t5/README.md (deflated 66%)
  adding: eu_ai_model_t5/training_args.bin (deflated 51%)
  adding: eu_ai_model_t5/adapter_model.safetensors (deflated 8%)
  adding: eu_ai_model_t5/special_tokens_map.json (deflated 85%)
  adding: eu_ai_model_t5/tokenizer_config.json (deflated 94%)


In [ ]:
import pickle

# Load the pickle file
with open('validation_dataset_july12.pkl', 'rb') as file:
    data = pickle.load(file)

# Extract questions from the data
questions = []
for item in data:
    if 'Questions' in item:
        questions.append(item['Questions'])

# Write questions to a text file
with open('Questions.txt', 'w', encoding='utf-8') as file:
    for question in questions:
        file.write(question + '\n')

print("Questions have been extracted and saved to Questions.txt")

Questions have been extracted and saved to Questions.txt


In [ ]:
import os
import time

def generate_response(prompt, max_length=520):
    """Generate response from the model for a given prompt."""
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    """
    With lora config one has to use **kwargs i.e key arguments such as input_ids=inputs["input_ids"]
    """
    outputs = model.generate(
                              input_ids=inputs["input_ids"],
                               max_length=max_length,
                              num_beams=5,
                              repetition_penalty=1.5,
                              temperature=1,
                              top_k=100,
                              top_p=0.5,
                              early_stopping=True
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def real_time_testing(*args):
    """
    Process either a single question or a text file of questions. For a single question, print the response.
    For a text file, save the questions and generated answers to a text file.

    Args:
        *args: Single question as a string or text file path as a string.
    """
    questions = []
    is_single_question = False

    # Check if the input is a single question or a file path
    if len(args) == 1 and isinstance(args[0], str):
        input_arg = args[0]
        if os.path.isfile(input_arg):  # Check if the input is a file path
            txt_filename = input_arg
            # Read questions from text file
            try:
                with open(txt_filename, 'r', encoding='utf-8') as txt_file:
                    questions = [line.strip() for line in txt_file.readlines() if line.strip()]
            except IOError as e:
                print(f"Error reading text file {txt_filename}: {e}")
                return
        else:  # Assume the input is a single question
            questions.append(input_arg)
            is_single_question = True
    else:
        print("Invalid input. Please provide a single question or a text file path.")
        return

    # Initialize variables to hold instruction-output pairs
    instruction_output_pairs = []

    start = time.time()

    # Generate outputs for each question and collect pairs
    for idx, question in enumerate(questions):
        generated_text = generate_response(question)
        instruction_output_pairs.append((question, generated_text))
        if is_single_question:
            print(f"*******************\n{idx + 1}. Prompt: {question} \nAnswer: {generated_text}\n")
        else:
            print(f"*******************\n{idx + 1}. Prompt: {question} \nAnswer: {generated_text}\n")

    end = time.time()
    print(f"Time taken: {end - start} seconds")

    if not is_single_question:
        # Define the output text file path
        output_txt_file = 'generated_outputs.txt'

        # Write instruction-output pairs to text file
        try:
            with open(output_txt_file, 'w', encoding='utf-8') as txt_file:
                for pair in instruction_output_pairs:
                    txt_file.write(f"Question: {pair[0]}\nAnswer: {pair[1]}\n\n")
            print(f"Text file with generated outputs saved: {output_txt_file}")
        except IOError as e:
            print(f"Error writing to text file {output_txt_file}: {e}")

In [ ]:
real_time_testing("Questions.txt")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:427: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `100` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


*******************
1. Prompt: What human oversight measures must be described? 
Answer: Under the EU AI Act, human oversight measures must be detailed as part of the overall human oversight process. This includes identifying and excluding human errors and biases. Providers must also ensure that human oversight is integrated into the AI system's operational lifecycle.

*******************
2. Prompt: How should transparency information be communicated to ensure it is actionable and meaningful? 
Answer: Transparency information must be communicated to ensure it is actionable and meaningful. Article 13 of the EU AI Act mandates that transparency information be made clear and accessible to all stakeholders. Providers should provide clear and accessible information about the AI system's functionality, decision-making processes, and any potential limitations or biases. They should also make it easy to understand and trust users, ensuring they can make informed decisions and validate the accu

In [ ]:
# push to the huggingface

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
trainer.push_to_hub('eu_ai_model_t5')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/7.12M [00:00<?, ?B/s]

Upload 6 LFS files:   0%|          | 0/6 [00:00<?, ?it/s]

events.out.tfevents.1720792149.520f718d09cb.681.1:   0%|          | 0.00/4.18k [00:00<?, ?B/s]

events.out.tfevents.1720792090.520f718d09cb.681.0:   0%|          | 0.00/4.18k [00:00<?, ?B/s]

events.out.tfevents.1720792208.520f718d09cb.1970.0:   0%|          | 0.00/28.6k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Orcawise/results/commit/5b97408bd1dd257011cdb250b56a133fa0e5bd47', commit_message='eu_ai_model_t5', commit_description='', oid='5b97408bd1dd257011cdb250b56a133fa0e5bd47', pr_url=None, pr_revision=None, pr_num=None)